In [1]:
import os
import sys
sys.path.append("../..")
import phage_init

In [2]:
#load the saved matrices
import pickle
train_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"train_x_data2_full.p"), "rb" ) )
test_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"test_x_data2_full.p"), "rb" ) )
train_Y=pickle.load(open( os.path.join(phage_init.data_dir,"train_y_data2_full.p"), "rb" ) )
test_Y=pickle.load(open( os.path.join(phage_init.data_dir,"test_y_data2_full.p"), "rb" ) )
mean_total=pickle.load(open( os.path.join(phage_init.data_dir,"mean_data2_full.p"), "rb" ) )
std_total=pickle.load(open( os.path.join(phage_init.data_dir,"std_data2_full.p"), "rb" ) )

In [3]:
di_train     = train_X_total[:,0:400]
tri_train    = train_X_total[:,400:8400]
di_sc_train  = train_X_total[:,8400:8449] 
tri_sc_train = train_X_total[:,8449:8792]
tt_train     = train_X_total[:,8792:]

In [4]:
di_test     = test_X_total[:,0:400]
tri_test    = test_X_total[:,400:8400]
di_sc_test  = test_X_total[:,8400:8449] 
tri_sc_test = test_X_total[:,8449:8792]
tt_test     = test_X_total[:,8792:]

In [5]:
mean_di     = mean_total[0:400]
mean_tri    = mean_total[400:8400]
mean_di_sc  = mean_total[8400:8449] 
mean_tri_sc = mean_total[8449:8792]
mean_tt     = mean_total[8792:]

In [6]:
std_di     = std_total[0:400]
std_tri    = std_total[400:8400]
std_di_sc  = std_total[8400:8449] 
std_tri_sc = std_total[8449:8792]
std_tt     = std_total[8792:]

In [7]:
#import keras and numpy
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [ ]:
#this list the devices, just making sure there is a GPU present, you might be fine with no GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [9]:
F = open(os.path.join(phage_init.model_dir,"10_cross.log"),'w') 

In [10]:
def get_formated_train(model_name):
    train_x=[]
    if model_name == 'di':
        train_x=di_train
    elif model_name == 'di_p':
        train_x=numpy.concatenate((di_train,tt_train),axis=1)
    elif model_name == 'tri':
        train_x=tri_train
    elif model_name == 'tri_p':
        train_x=numpy.concatenate((tri_train,tt_train),axis=1)
    elif model_name == 'di_sc':
        train_x=di_sc_train
    elif model_name == 'di_sc_p':
        train_x=numpy.concatenate((di_sc_train,tt_train),axis=1)
    elif model_name == 'tri_sc':
        train_x=tri_sc_train
    elif model_name == 'tri_sc_p':
        train_x=numpy.concatenate((tri_sc_train,tt_train),axis=1)
    elif model_name == 'all':
        train_x=train_X_total
    return train_x

In [13]:
from sklearn.model_selection import StratifiedKFold
def train_kfold(model_name):
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=43)
    train_Y_index = train_Y.argmax(axis=1)
    cvscores = []
    model_number=1
    train_X=get_formated_train(model_name=model_name)
    f_num=train_X.shape[1]
    print("Doing cross validation on "+model_name)
    F.write("Doing cross validation on "+model_name)
    for train, test in kfold.split(train_X, train_Y_index):
    #model with 2 leyers of 100 LSTM neurons
        train_YY = numpy.eye(10)[train_Y_index[train]]
        test_YY  = numpy.eye(10)[train_Y_index[test]]
        model = Sequential()
        opt=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.add(Dense(f_num, input_dim=f_num, kernel_initializer='random_uniform',activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(200,activation='relu'))
    #model.add(Dense(200,activation='sigmoid'))
        model.add(Dropout(0.2))
        model.add(Dense(200,activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(10,activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        model.fit(train_X[train], train_YY, epochs=30, batch_size=3000, verbose=0)
        scores = model.evaluate(train_X[test], test_YY, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        F.write("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
        model.save( os.path.join(phage_init.model_dir,model_name+'_'+"{:02d}".format(model_number)+'.h5') )
        pickle.dump(train_X[test], open( os.path.join(phage_init.data_dir, model_name+'_'+"{:02d}".format(model_number)+'_test_X.p' ), "wb"))
        pickle.dump(test_YY, open( os.path.join(phage_init.data_dir, model_name+'_'+"{:02d}".format(model_number)+'_test_Y.p' ), "wb"))
        model_number=model_number+1
        del model
        F.flush()
    print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))  
    F.write("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
    F.flush()

In [ ]:
all_models=['di','di_p','tri','tri_p','di_sc','di_sc_p','tri_sc','tri_sc_p','all']
for this_model in all_models:
    train_kfold(this_model)
    

Doing cross validation on di
acc: 91.29%
acc: 91.05%
acc: 91.20%
acc: 91.38%
acc: 91.30%
acc: 91.30%
acc: 89.84%
acc: 89.31%
acc: 89.59%
acc: 88.91%
90.52% (+/- 0.93%)
Doing cross validation on di_p
acc: 91.67%
acc: 91.39%
acc: 91.88%
acc: 91.74%
acc: 91.20%
acc: 91.84%
acc: 91.52%
acc: 90.47%
acc: 90.27%
acc: 90.63%
91.26% (+/- 0.57%)
Doing cross validation on tri
acc: 92.73%


In [ ]:
F.close()